# Timing Diagrams in Digital Design

*Understanding signal behavior in multi-component systems*

---

## Prerequisites

This tutorial assumes you have completed:

- **[D Flip-Flops in Digital Design](/d_flipflop_tutorial.html)** — including how to read basic timing diagrams, edge-triggered capture, setup/hold times, and propagation delay

You should also be familiar with:

- **Binary numbers** and logic levels (HIGH/LOW, 1/0)
- **Logic gates** (AND, OR, NOT) and their behavior

---

## What This Tutorial Covers

The D flip-flop tutorial taught you to read timing diagrams for a single component. This tutorial extends that skill to **system-level timing diagrams** involving multiple signals working together:

- **Signal types and notation** — buses, active-low signals, conventions
- **Input/output perspectives** — understanding whose view you're looking at
- **Multi-signal coordination** — memory interfaces, handshaking protocols
- **Pipeline timing** — overlapped operations
- **Timing hazards** — glitches and how to avoid them

These skills are essential for reading datasheets and debugging real hardware.

---

## Anatomy of a Timing Diagram

Let's start with a simple timing diagram showing a clock and a data signal.

**About the example data:** The data pattern `[0, 0, 1, 1, 1, 0, 1, 0]` was chosen deliberately to show:
- **Staying low** (cycles 0-1): Data can remain stable across multiple cycles
- **Rising transition** (cycle 2): LOW→HIGH change
- **Staying high** (cycles 2-4): Data holds its value
- **Multiple transitions** (cycles 5-7): Demonstrates that data can change every cycle

This pattern gives you a representative sample of the behaviors you'll encounter in real timing diagrams.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(2, 1, figsize=(12, 4), sharex=True)
fig.suptitle('Basic Timing Diagram Components', fontsize=14, fontweight='bold')

t = np.arange(0, 8, 0.01)

# Clock signal
clk = np.zeros_like(t)
for i in range(8):
    clk[(t >= i) & (t < i + 0.5)] = 1

# Data signal
data_values = [0, 0, 1, 1, 1, 0, 1, 0]
data = np.zeros_like(t)
for i, val in enumerate(data_values):
    data[(t >= i) & (t < i + 1)] = val

# Plot clock
axes[0].fill_between(t, 0, clk, color='#3b82f6', alpha=0.3, step='pre')
axes[0].plot(t, clk, color='#3b82f6', lw=2, drawstyle='steps-pre')
axes[0].set_ylabel('CLK', fontsize=11, fontweight='bold')
axes[0].set_ylim(-0.3, 1.5)
axes[0].set_yticks([0, 1])
axes[0].set_yticklabels(['0', '1'])
axes[0].grid(True, alpha=0.3, axis='x')

# Annotate clock features
axes[0].annotate('Rising edge', xy=(1.0, 0.5), xytext=(1.2, 1.3),
                fontsize=9, color='#1e40af',
                arrowprops=dict(arrowstyle='->', color='#1e40af', lw=1.5))
axes[0].annotate('Falling edge', xy=(1.5, 0.5), xytext=(1.8, -0.1),
                fontsize=9, color='#1e40af',
                arrowprops=dict(arrowstyle='->', color='#1e40af', lw=1.5))
axes[0].annotate('', xy=(3, 1.35), xytext=(4, 1.35),
                arrowprops=dict(arrowstyle='<->', color='#059669', lw=1.5))
axes[0].text(3.5, 1.45, 'Period', ha='center', fontsize=9, color='#059669')

# Plot data
axes[1].fill_between(t, 0, data, color='#10b981', alpha=0.3, step='pre')
axes[1].plot(t, data, color='#10b981', lw=2, drawstyle='steps-pre')
axes[1].set_ylabel('DATA', fontsize=11, fontweight='bold')
axes[1].set_ylim(-0.3, 1.5)
axes[1].set_yticks([0, 1])
axes[1].set_yticklabels(['0', '1'])
axes[1].set_xlabel('Time', fontsize=11)
axes[1].grid(True, alpha=0.3, axis='x')

# Mark clock edges
for i in range(8):
    for ax in axes:
        ax.axvline(x=i, color='#d1d5db', linestyle='--', lw=0.5)

plt.tight_layout()
plt.show()

**Key elements:**

| Element | Description |
|---------|-------------|
| **Signal name** | Label on Y-axis (CLK, DATA, etc.) |
| **Logic levels** | HIGH (1) and LOW (0) positions |
| **Time axis** | Horizontal axis showing progression |
| **Rising edge** | Transition from LOW to HIGH (↑) |
| **Falling edge** | Transition from LOW to HIGH (↓) |
| **Period** | Time for one complete clock cycle |

---

## Signal Types in Timing Diagrams

Digital systems use several types of signals, each with distinct characteristics.

**Important: Inputs vs Outputs**

Timing diagrams show signals from the perspective of a specific component. Before reading any timing diagram, ask: "What component am I looking at?"

- **Inputs**: Signals coming *into* the component from external sources
- **Outputs**: Signals generated *by* the component

In the example below, imagine we're looking at a **data processing module**:

| Signal | Direction | Source |
|--------|-----------|--------|
| CLK | Input | Crystal oscillator or PLL on the board |
| RST_N | Input | Power-on reset circuit (hardware that monitors power supply) |
| EN | Input | A controller or state machine that decides when this module should operate |
| D | Input | Upstream logic (e.g., a FIFO, another module's output, or external pins) |
| BUS[7:0] | Output | This module's result — downstream logic will read it |

**About the example signals:**

- **CLK**: Input from the system clock generator — every synchronous component receives this

- **RST_N (active-low reset)**: Input from the **power-on reset circuit**. The timing (0.3 to 1.8) represents:
  - **Starts at 0.3** (not exactly 0): The reset circuit needs time to detect stable power
  - **Releases at 1.8**: Released after power and clock are confirmed stable
  - This is *asynchronous* to the clock — notice it doesn't align with clock edges

- **EN (enable)**: Input from a **controller state machine** that orchestrates the system. The controller asserts EN when it wants this module to process data (cycles 2-7), then deasserts it when done.

- **D (data)**: Input from **upstream logic** — could be a sensor interface, a FIFO buffer, or another module's output. The pattern shows data arriving while the module is enabled.

- **BUS[7:0]**: Output produced by this module. The **slanting "X" lines at transitions** are standard notation meaning:
  - All 8 bits are changing simultaneously
  - The "X" pattern says "transition happening — don't sample during this time"
  - Values shown (`XX`, `00`, `01`, etc.) are the stable values between transitions
  - `XX` at startup means "unknown" — the module hasn't produced valid output yet

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(5, 1, figsize=(12, 10), sharex=True)
fig.suptitle('Common Signal Types', fontsize=14, fontweight='bold')

t = np.arange(0, 10, 0.01)

# 1. Clock signal - periodic
clk = np.zeros_like(t)
for i in range(10):
    clk[(t >= i) & (t < i + 0.5)] = 1

axes[0].fill_between(t, 0, clk, color='#3b82f6', alpha=0.3, step='pre')
axes[0].plot(t, clk, color='#3b82f6', lw=2, drawstyle='steps-pre')
axes[0].set_ylabel('CLK', fontsize=10, fontweight='bold')
axes[0].set_ylim(-0.2, 1.4)
axes[0].set_yticks([0, 1])
axes[0].text(10.2, 0.5, 'Clock\n(periodic)', fontsize=9, va='center', color='#3b82f6')

# 2. Reset signal - active low, asynchronous
reset = np.ones_like(t)
reset[(t >= 0.3) & (t < 1.8)] = 0

axes[1].fill_between(t, 0, reset, color='#ef4444', alpha=0.3, step='pre')
axes[1].plot(t, reset, color='#ef4444', lw=2, drawstyle='steps-pre')
axes[1].set_ylabel('RST_N', fontsize=10, fontweight='bold')
axes[1].set_ylim(-0.2, 1.4)
axes[1].set_yticks([0, 1])
axes[1].text(10.2, 0.5, 'Reset\n(active-low)', fontsize=9, va='center', color='#ef4444')
axes[1].annotate('Active', xy=(1, 0.2), fontsize=8, color='#ef4444', ha='center')

# 3. Enable signal - control
enable = np.zeros_like(t)
enable[(t >= 2) & (t < 7)] = 1

axes[2].fill_between(t, 0, enable, color='#f59e0b', alpha=0.3, step='pre')
axes[2].plot(t, enable, color='#f59e0b', lw=2, drawstyle='steps-pre')
axes[2].set_ylabel('EN', fontsize=10, fontweight='bold')
axes[2].set_ylim(-0.2, 1.4)
axes[2].set_yticks([0, 1])
axes[2].text(10.2, 0.5, 'Enable\n(control)', fontsize=9, va='center', color='#f59e0b')

# 4. Data signal - changes on clock edges
data_values = [0, 0, 0, 1, 0, 1, 1, 0, 1, 0]
data = np.zeros_like(t)
for i, val in enumerate(data_values):
    data[(t >= i) & (t < i + 1)] = val

axes[3].fill_between(t, 0, data, color='#10b981', alpha=0.3, step='pre')
axes[3].plot(t, data, color='#10b981', lw=2, drawstyle='steps-pre')
axes[3].set_ylabel('D', fontsize=10, fontweight='bold')
axes[3].set_ylim(-0.2, 1.4)
axes[3].set_yticks([0, 1])
axes[3].text(10.2, 0.5, 'Data\n(synchronous)', fontsize=9, va='center', color='#10b981')

# 5. Bus signal - multi-bit represented as a group
bus_values = ['XX', '00', '00', '01', '02', '03', '03', '03', '04', '00']
bus = np.zeros_like(t)
for i in range(10):
    bus[(t >= i) & (t < i + 1)] = 0.5

axes[4].fill_between(t, 0.2, 0.8, color='#8b5cf6', alpha=0.2, step='pre')
axes[4].plot(t, np.ones_like(t) * 0.8, color='#8b5cf6', lw=2, drawstyle='steps-pre')
axes[4].plot(t, np.ones_like(t) * 0.2, color='#8b5cf6', lw=2, drawstyle='steps-pre')
# Draw transitions
for i in range(1, 10):
    if bus_values[i] != bus_values[i-1]:
        axes[4].plot([i, i], [0.2, 0.8], color='#8b5cf6', lw=2)
        # X crossing for transition
        axes[4].plot([i-0.05, i+0.05], [0.2, 0.8], color='#8b5cf6', lw=1)
        axes[4].plot([i-0.05, i+0.05], [0.8, 0.2], color='#8b5cf6', lw=1)
for i, val in enumerate(bus_values):
    axes[4].text(i + 0.5, 0.5, val, ha='center', va='center', fontsize=9, fontweight='bold', color='#5b21b6')

axes[4].set_ylabel('BUS[7:0]', fontsize=10, fontweight='bold')
axes[4].set_ylim(-0.1, 1.1)
axes[4].set_yticks([])
axes[4].set_xlabel('Clock Cycles', fontsize=11)
axes[4].text(10.2, 0.5, 'Bus\n(multi-bit)', fontsize=9, va='center', color='#8b5cf6')

# Mark clock edges
for i in range(10):
    for ax in axes:
        ax.axvline(x=i, color='#d1d5db', linestyle='--', lw=0.5)

plt.tight_layout()
plt.show()

**Signal type characteristics:**

| Type | Characteristics | Examples |
|------|-----------------|----------|
| **Clock** | Periodic, fixed frequency | CLK, SYSCLK |
| **Reset** | Often active-low, assertion clears state | RST_N, RESET |
| **Enable** | Gates operations on/off | EN, CE, OE |
| **Data** | Changes relative to clock | D, Q, DATA_IN |
| **Bus** | Multi-bit values shown as hex/decimal | ADDR[15:0], DATA[7:0] |

---

## Example: Reading a Memory Interface

Let's examine a realistic timing diagram for a simple synchronous memory read operation.

**About the signal timing:** This example models a typical synchronous SRAM read sequence:

- **CS_N (Chip Select)**: Goes low at cycle 2 and stays low through cycle 6 — the memory chip ignores all other signals when CS_N is high
- **RD_N (Read Enable)**: Asserted one cycle after CS_N (cycle 3) — this is the actual read command
- **ADDR**: Set to address `00` when CS_N goes low — we're reading from memory address 0x00
- **DATA**: Returns `42` after one cycle delay (cycle 4) — memory needs time to fetch the data
- **VALID**: Goes high when data is ready — tells the controller "you can sample now"

**Why these specific values?**
- Address `00` is simple — any address would work the same way
- Data value `42` (the "answer to everything") is memorable and clearly shows valid data vs high-impedance `ZZ`
- One-cycle latency between RD_N assertion and data valid is typical for synchronous memory
- The `ZZ` (high-impedance) shows the data bus is not being driven when no read is active

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(6, 1, figsize=(14, 10), sharex=True)
fig.suptitle('Synchronous Memory Read Timing', fontsize=14, fontweight='bold')

t = np.arange(0, 10, 0.01)

# Clock
clk = np.zeros_like(t)
for i in range(10):
    clk[(t >= i) & (t < i + 0.5)] = 1

# Chip Select (active low) - asserted at cycle 2
cs_n = np.ones_like(t)
cs_n[(t >= 2) & (t < 7)] = 0

# Read Enable (active low) - asserted at cycle 3
rd_n = np.ones_like(t)
rd_n[(t >= 3) & (t < 6)] = 0

# Address bus
addr_values = ['--', '--', '00', '00', '00', '00', '00', '--', '--', '--']

# Data bus - valid 1 cycle after read enable
data_values = ['ZZ', 'ZZ', 'ZZ', 'ZZ', '42', '42', 'ZZ', 'ZZ', 'ZZ', 'ZZ']

# Data Valid signal
valid = np.zeros_like(t)
valid[(t >= 4) & (t < 6)] = 1

# Plot clock
axes[0].fill_between(t, 0, clk, color='#3b82f6', alpha=0.3, step='pre')
axes[0].plot(t, clk, color='#3b82f6', lw=2, drawstyle='steps-pre')
axes[0].set_ylabel('CLK', fontsize=10, fontweight='bold')
axes[0].set_ylim(-0.2, 1.4)
axes[0].set_yticks([0, 1])

# Plot CS_N
axes[1].fill_between(t, 0, cs_n, color='#ef4444', alpha=0.3, step='pre')
axes[1].plot(t, cs_n, color='#ef4444', lw=2, drawstyle='steps-pre')
axes[1].set_ylabel('CS_N', fontsize=10, fontweight='bold')
axes[1].set_ylim(-0.2, 1.4)
axes[1].set_yticks([0, 1])
axes[1].text(4.5, 0.2, 'chip selected', fontsize=9, ha='center', color='#ef4444')

# Plot RD_N
axes[2].fill_between(t, 0, rd_n, color='#f59e0b', alpha=0.3, step='pre')
axes[2].plot(t, rd_n, color='#f59e0b', lw=2, drawstyle='steps-pre')
axes[2].set_ylabel('RD_N', fontsize=10, fontweight='bold')
axes[2].set_ylim(-0.2, 1.4)
axes[2].set_yticks([0, 1])
axes[2].text(4.5, 0.2, 'read active', fontsize=9, ha='center', color='#f59e0b')

# Plot Address bus
axes[3].fill_between(t, 0.2, 0.8, color='#8b5cf6', alpha=0.2)
axes[3].plot(t, np.ones_like(t) * 0.8, color='#8b5cf6', lw=2)
axes[3].plot(t, np.ones_like(t) * 0.2, color='#8b5cf6', lw=2)
for i in range(1, 10):
    if addr_values[i] != addr_values[i-1]:
        axes[3].plot([i, i], [0.2, 0.8], color='#8b5cf6', lw=2)
        axes[3].plot([i-0.05, i+0.05], [0.2, 0.8], color='#8b5cf6', lw=1)
        axes[3].plot([i-0.05, i+0.05], [0.8, 0.2], color='#8b5cf6', lw=1)
for i, val in enumerate(addr_values):
    axes[3].text(i + 0.5, 0.5, val, ha='center', va='center', fontsize=9, fontweight='bold', color='#5b21b6')
axes[3].set_ylabel('ADDR', fontsize=10, fontweight='bold')
axes[3].set_ylim(-0.1, 1.1)
axes[3].set_yticks([])

# Plot Data bus
axes[4].fill_between(t, 0.2, 0.8, color='#10b981', alpha=0.2)
axes[4].plot(t, np.ones_like(t) * 0.8, color='#10b981', lw=2)
axes[4].plot(t, np.ones_like(t) * 0.2, color='#10b981', lw=2)
for i in range(1, 10):
    if data_values[i] != data_values[i-1]:
        axes[4].plot([i, i], [0.2, 0.8], color='#10b981', lw=2)
        axes[4].plot([i-0.05, i+0.05], [0.2, 0.8], color='#10b981', lw=1)
        axes[4].plot([i-0.05, i+0.05], [0.8, 0.2], color='#10b981', lw=1)
for i, val in enumerate(data_values):
    color = '#065f46' if val not in ['ZZ', '--'] else '#9ca3af'
    axes[4].text(i + 0.5, 0.5, val, ha='center', va='center', fontsize=9, fontweight='bold', color=color)
axes[4].set_ylabel('DATA', fontsize=10, fontweight='bold')
axes[4].set_ylim(-0.1, 1.1)
axes[4].set_yticks([])

# Plot Valid
axes[5].fill_between(t, 0, valid, color='#22c55e', alpha=0.3, step='pre')
axes[5].plot(t, valid, color='#22c55e', lw=2, drawstyle='steps-pre')
axes[5].set_ylabel('VALID', fontsize=10, fontweight='bold')
axes[5].set_ylim(-0.2, 1.4)
axes[5].set_yticks([0, 1])
axes[5].set_xlabel('Clock Cycles', fontsize=11)
axes[5].text(5, 1.2, 'Data can be sampled', fontsize=9, ha='center', color='#22c55e')

# Mark clock edges
for i in range(10):
    for ax in axes:
        ax.axvline(x=i, color='#d1d5db', linestyle='--', lw=0.5)

# Annotate phases
axes[0].text(2.5, 1.3, '1. Select chip', fontsize=9, ha='center', color='#374151')
axes[0].text(3.5, 1.3, '2. Assert read', fontsize=9, ha='center', color='#374151')
axes[0].text(5, 1.3, '3. Capture data', fontsize=9, ha='center', color='#374151')

plt.tight_layout()
plt.show()

**Reading this timing diagram:**

1. **Cycle 2**: Address is placed on bus, chip select asserted (CS_N goes low)
2. **Cycle 3**: Read enable asserted (RD_N goes low)
3. **Cycle 4**: Memory responds with data, VALID goes high
4. **Cycles 4-5**: Data is valid — controller samples it on the rising edge
5. **Cycle 6**: Read complete, signals deasserted

**Notation conventions:**
- **ZZ**: High-impedance (tri-state, bus not driven)
- **--**: Don't care (value irrelevant)
- **_N suffix**: Active-low signal

---

## Common Timing Diagram Patterns

### Pattern 1: Handshaking (Request/Acknowledge)

**About the signal timing:** This pattern shows a classic "four-phase handshake":

1. **REQ asserts** (cycle 2): Requester says "I need something"
2. **ACK asserts** (cycle 4): Responder says "Got it, working on it" — the 2-cycle delay represents processing time
3. **REQ deasserts** (cycle 6): Requester sees ACK, says "Okay, I'll wait"
4. **ACK deasserts** (cycle 7): Responder finishes, ready for next request

**Why these specific timings?**
- 2-cycle delay before ACK: Shows that the responder needs time to process (could be memory access, computation, etc.)
- REQ held until ACK seen: Demonstrates proper protocol — you don't drop your request until acknowledged
- ACK held one cycle after REQ drops: Shows clean completion of the handshake

This pattern is fundamental — you'll see it in bus protocols, FIFO interfaces, and inter-module communication.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(3, 1, figsize=(12, 5), sharex=True)
fig.suptitle('Request/Acknowledge Handshaking', fontsize=14, fontweight='bold')

t = np.arange(0, 10, 0.01)

# Clock
clk = np.zeros_like(t)
for i in range(10):
    clk[(t >= i) & (t < i + 0.5)] = 1

# Request
req = np.zeros_like(t)
req[(t >= 2) & (t < 6)] = 1

# Acknowledge (delayed response)
ack = np.zeros_like(t)
ack[(t >= 4) & (t < 7)] = 1

# Plot
axes[0].fill_between(t, 0, clk, color='#3b82f6', alpha=0.3, step='pre')
axes[0].plot(t, clk, color='#3b82f6', lw=2, drawstyle='steps-pre')
axes[0].set_ylabel('CLK', fontsize=10, fontweight='bold')
axes[0].set_ylim(-0.2, 1.4)
axes[0].set_yticks([0, 1])

axes[1].fill_between(t, 0, req, color='#10b981', alpha=0.3, step='pre')
axes[1].plot(t, req, color='#10b981', lw=2, drawstyle='steps-pre')
axes[1].set_ylabel('REQ', fontsize=10, fontweight='bold')
axes[1].set_ylim(-0.2, 1.4)
axes[1].set_yticks([0, 1])

axes[2].fill_between(t, 0, ack, color='#f59e0b', alpha=0.3, step='pre')
axes[2].plot(t, ack, color='#f59e0b', lw=2, drawstyle='steps-pre')
axes[2].set_ylabel('ACK', fontsize=10, fontweight='bold')
axes[2].set_ylim(-0.2, 1.4)
axes[2].set_yticks([0, 1])
axes[2].set_xlabel('Clock Cycles', fontsize=11)

# Annotations
axes[1].annotate('Request asserted', xy=(2.5, 1.1), fontsize=9, color='#10b981')
axes[2].annotate('Acknowledge returned', xy=(4.5, 1.1), fontsize=9, color='#f59e0b')
axes[1].annotate('Request deasserted\nafter ACK seen', xy=(6.5, 0.6), fontsize=9, color='#6b7280')

for i in range(10):
    for ax in axes:
        ax.axvline(x=i, color='#d1d5db', linestyle='--', lw=0.5)

plt.tight_layout()
plt.show()

### Pattern 2: Pipeline Stages

**About the example:** We show 5 instructions (A, B, C, D, E) flowing through a 3-stage pipeline (Fetch → Decode → Execute).

**Why these specific values?**
- **3 stages**: The minimum to show pipeline behavior clearly — real CPUs have 5-20+ stages, but the principle is the same
- **5 instructions**: Enough to show:
  - Pipeline filling (cycles 1-3): Each cycle adds one more instruction in flight
  - Steady state (cycles 4-5): All 3 stages busy simultaneously
  - Pipeline draining (cycles 6-7): Last instructions completing
- **Letters A-E**: Simple labels that make it easy to track each instruction's progress through stages

**Key insight to notice:** Look at cycle 5 in the diagram — at that single instant:
- Instruction **C** is being fetched
- Instruction **B** is being decoded
- Instruction **A** is executing

This "overlap" is why pipelines improve throughput — we're doing 3 things at once instead of waiting for each instruction to fully complete.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(4, 1, figsize=(14, 7), sharex=True)
fig.suptitle('3-Stage Pipeline Timing', fontsize=14, fontweight='bold')

t = np.arange(0, 12, 0.01)

# Clock
clk = np.zeros_like(t)
for i in range(12):
    clk[(t >= i) & (t < i + 0.5)] = 1

# Define colors for each instruction
colors = ['#3b82f6', '#10b981', '#f59e0b', '#ef4444', '#8b5cf6']
instr_names = ['A', 'B', 'C', 'D', 'E']

# Stage 1: Fetch
# Stage 2: Decode (1 cycle after fetch)
# Stage 3: Execute (2 cycles after fetch)

def draw_pipeline_stage(ax, stage_data, ylabel):
    ax.fill_between(t, 0.2, 0.8, color='#f3f4f6', alpha=0.5)
    ax.plot(t, np.ones_like(t) * 0.8, color='#9ca3af', lw=1)
    ax.plot(t, np.ones_like(t) * 0.2, color='#9ca3af', lw=1)
    
    for start, name, color in stage_data:
        ax.fill_between(t[(t >= start) & (t < start + 1)], 0.2, 0.8, color=color, alpha=0.6, step='pre')
        ax.text(start + 0.5, 0.5, name, ha='center', va='center', fontsize=11, fontweight='bold', color='white')
        ax.axvline(x=start, color='#374151', lw=1)
        ax.axvline(x=start+1, color='#374151', lw=1)
    
    ax.set_ylabel(ylabel, fontsize=10, fontweight='bold')
    ax.set_ylim(-0.1, 1.1)
    ax.set_yticks([])

# Plot clock
axes[0].fill_between(t, 0, clk, color='#3b82f6', alpha=0.3, step='pre')
axes[0].plot(t, clk, color='#3b82f6', lw=2, drawstyle='steps-pre')
axes[0].set_ylabel('CLK', fontsize=10, fontweight='bold')
axes[0].set_ylim(-0.2, 1.4)
axes[0].set_yticks([0, 1])

# Fetch stage
fetch_data = [(1, 'A', colors[0]), (2, 'B', colors[1]), (3, 'C', colors[2]), 
              (4, 'D', colors[3]), (5, 'E', colors[4])]
draw_pipeline_stage(axes[1], fetch_data, 'FETCH')

# Decode stage (shifted by 1)
decode_data = [(2, 'A', colors[0]), (3, 'B', colors[1]), (4, 'C', colors[2]),
               (5, 'D', colors[3]), (6, 'E', colors[4])]
draw_pipeline_stage(axes[2], decode_data, 'DECODE')

# Execute stage (shifted by 2)
exec_data = [(3, 'A', colors[0]), (4, 'B', colors[1]), (5, 'C', colors[2]),
             (6, 'D', colors[3]), (7, 'E', colors[4])]
draw_pipeline_stage(axes[3], exec_data, 'EXECUTE')
axes[3].set_xlabel('Clock Cycles', fontsize=11)

# Mark clock edges
for i in range(12):
    for ax in axes:
        ax.axvline(x=i, color='#d1d5db', linestyle='--', lw=0.5)

# Annotations
axes[0].text(6.5, 1.3, 'At cycle 5: A executing, B decoding, C fetching', fontsize=10, color='#374151')

plt.tight_layout()
plt.show()

**Pipeline insight:** At any given cycle, multiple instructions are in flight simultaneously:
- Each stage processes a different instruction
- One instruction completes per cycle (after the pipeline fills)
- Latency = 3 cycles, but throughput = 1 instruction/cycle

---

## Timing Hazards and Issues

### Glitches

A **glitch** is an unwanted short pulse caused by unequal propagation delays through different logic paths.

**About this example:** We use the simple logic function `Y = A AND (NOT B)` with a specific input change designed to expose the glitch:

**Why A=0→1 and B=0→1 simultaneously?**
- In the ideal world (zero delay): When both go from 0 to 1:
  - A becomes 1, B becomes 1, so NOT B becomes 0
  - Y = 1 AND 0 = 0 (no change from initial state)
- In the real world: The NOT gate has a small delay (~0.1 time units in our example)
  - A rises to 1 immediately
  - NOT B is still 1 (hasn't processed B's change yet)
  - For that brief moment: Y = 1 AND 1 = 1 → **GLITCH!**

**Why this matters:** This specific input transition (both inputs changing together) is the classic "static hazard" case. The glitch duration equals the NOT gate's propagation delay. In a real circuit, this could:
- Cause a downstream flip-flop to capture the wrong value
- Trigger unintended state machine transitions
- Corrupt data in asynchronous designs

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(4, 1, figsize=(12, 6), sharex=True)
fig.suptitle('Glitch in Combinational Logic (Y = A AND NOT B)', fontsize=14, fontweight='bold')

t = np.arange(0, 6, 0.001)

# A changes at t=2
a_signal = np.zeros_like(t)
a_signal[t >= 2] = 1

# B changes at t=2 (simultaneously in ideal world)
b_signal = np.zeros_like(t)
b_signal[t >= 2] = 1

# NOT B (with small delay of 0.1)
not_b = np.ones_like(t)
not_b[t >= 2.1] = 0

# Y = A AND NOT_B - shows glitch because A rises before NOT_B falls
y_signal = np.zeros_like(t)
y_signal[(t >= 2) & (t < 2.1)] = 1  # Glitch!

# Plot
axes[0].fill_between(t, 0, a_signal, color='#3b82f6', alpha=0.3, step='pre')
axes[0].plot(t, a_signal, color='#3b82f6', lw=2, drawstyle='steps-pre')
axes[0].set_ylabel('A', fontsize=11, fontweight='bold')
axes[0].set_ylim(-0.2, 1.4)
axes[0].set_yticks([0, 1])

axes[1].fill_between(t, 0, b_signal, color='#10b981', alpha=0.3, step='pre')
axes[1].plot(t, b_signal, color='#10b981', lw=2, drawstyle='steps-pre')
axes[1].set_ylabel('B', fontsize=11, fontweight='bold')
axes[1].set_ylim(-0.2, 1.4)
axes[1].set_yticks([0, 1])

axes[2].fill_between(t, 0, not_b, color='#f59e0b', alpha=0.3, step='pre')
axes[2].plot(t, not_b, color='#f59e0b', lw=2, drawstyle='steps-pre')
axes[2].set_ylabel('NOT B', fontsize=11, fontweight='bold')
axes[2].set_ylim(-0.2, 1.4)
axes[2].set_yticks([0, 1])
axes[2].annotate('Delayed!', xy=(2.1, 0.5), xytext=(2.5, 1.1),
                fontsize=9, color='#f59e0b',
                arrowprops=dict(arrowstyle='->', color='#f59e0b'))

axes[3].fill_between(t, 0, y_signal, color='#ef4444', alpha=0.3, step='pre')
axes[3].plot(t, y_signal, color='#ef4444', lw=2, drawstyle='steps-pre')
axes[3].set_ylabel('Y', fontsize=11, fontweight='bold')
axes[3].set_ylim(-0.2, 1.4)
axes[3].set_yticks([0, 1])
axes[3].set_xlabel('Time', fontsize=11)

# Highlight glitch
axes[3].axvspan(2, 2.1, color='#fecaca', alpha=0.7)
axes[3].annotate('GLITCH!', xy=(2.05, 1.0), xytext=(2.5, 1.2),
                fontsize=10, fontweight='bold', color='#ef4444',
                arrowprops=dict(arrowstyle='->', color='#ef4444', lw=2))

axes[0].axvline(x=2, color='#d1d5db', linestyle='--', lw=1)

plt.tight_layout()
plt.show()

**Why glitches occur:** When A and B both change from 0→1:
- Ideally: Y stays 0 (since A·B̄ = 1·0 = 0)
- Reality: A rises immediately, but NOT B takes time to fall
- Brief moment: A=1, NOT B=1 (hasn't updated yet) → Y=1 (glitch!)

**Avoiding glitch problems:**
- Register outputs to sample only at clock edges
- Use synchronous design practices
- Add glitch filters for asynchronous inputs

---

## Key Takeaways

1. **Timing diagrams are essential** for understanding and debugging digital circuits

2. **Clock edges are reference points** — in synchronous design, everything happens relative to the clock

3. **Setup and hold times** define when data must be stable for reliable capture

4. **Propagation delay** limits how fast your circuit can run

5. **Common notation:**
   - Active-low signals: _N suffix or overbar
   - High-impedance: Z or Hi-Z
   - Don't care: X or --
   - Bus transitions: X-crossing pattern

6. **Practice reading timing diagrams** from datasheets — they're the universal language of digital hardware

---

*Timing diagrams bridge the gap between abstract logic design and physical hardware reality. Master them, and you'll be able to debug the most challenging timing issues.*